Nikhil Adithyan
Aug 1, 2023 /7 min read

I’ve always been a huge fan of TradingView’s charting tool, especially for its beautifully crafted user interface and design. And there’s never been a day I haven’t thought about recreating the graph design in Python.
But, it’s always tough for Python developers to create stunning and professional-looking visualizations (like TradingView) using libraries like Matplotlib, Seaborn, Altair, etc. Their style themes are so outdated and crappy. Though all these modules provide features for customizing the theme of the charts, it takes a toll on the developer as there is a lot of work involved.


Fortunately, I recently came across an awesome library called lightweight-charts-python providing features to easily re-create the TradingView style with minimal code. In this article, we will dive deep into this library, explore its features, and code some cool TradingView charts in Python.

Importing Packages
The first and foremost step of setting up the coding environment is to import the required packages. In this article, we are going to use five different packages which are pandas for data manipulation, and requests for making API calls, numpy for numerical calculations, lightweight_chart for replicating the TradingView look, time for time-related functions, and finally asyncio and nest_asyncio for asynchronous programming. The following code will import all the mentioned packages into our Python environment:

In [1]:
import pandas as pd
import requests
import numpy as np
from lightweight_charts import Chart
from stock_indicators import indicators, Quote
import time
import asyncio
import nest_asyncio
nest_asyncio.apply()

Obtaining Data using Yfinance

In [2]:
import yfinance as yf
df = yf.download('TQQQ', start='2020-01-01', multi_level_index=False)
df.reset_index(inplace=True)

df.head()

C:\Users\jwang\AppData\Local\Temp\ipykernel_22036\3415087877.py:2: FutureWarning: YF.download() has changed argument auto_adjust default to True
  df = yf.download('TQQQ', start='2020-01-01', multi_level_index=False)
[*********************100%***********************]  1 of 1 completed


,Date,Close,High,Low,Open,Volume
0,2020-01-02,21.760864,21.760864,21.133240,21.250620,65536000
1,2020-01-03,21.169174,21.559643,20.898480,20.910458,72590000
2,2020-01-06,21.571619,21.578805,20.589456,20.666113,64047600
3,2020-01-07,21.545265,21.746488,21.358415,21.593175,53849600
4,2020-01-08,22.041138,22.345367,21.447049,21.550056,79582400


In [3]:
quotes = [
    Quote(d, o, h, l, c, v)
    for d, o, h, l, c, v in zip(
        df['Date'],
        df['Open'],
        df['High'],
        df['Low'],
        df['Close'],
        df['Volume']
    )
]

In [4]:
ema_results_25 = indicators.get_ema(quotes, 25)
df['EMA 25'] = [r.ema for r in ema_results_25]
df['EMA 12'] = [r.ema for r in indicators.get_ema(quotes, 12)]
df['upper_band'] = [r.upper_band for r in indicators.get_bollinger_bands(quotes, 20, 2)]
df['middle_band'] = [r.sma for r in indicators.get_bollinger_bands(quotes, 20, 2)]
df['lower_band'] = [r.lower_band for r in indicators.get_bollinger_bands(quotes, 20, 2)]
df['rsi'] = [r.rsi for r in indicators.get_rsi(quotes, 14)]
df = df.dropna().reset_index(drop=True)
df.head()

,Date,Close,High,Low,Open,Volume,EMA 25,EMA 12,upper_band,middle_band,lower_band,rsi
0,2020-02-06,25.972179,26.000926,25.251131,25.514637,56566400,23.212258,24.073253,25.548904,23.610919,21.672935,69.315864
1,2020-02-07,25.636805,26.065602,25.421210,25.627224,77026000,23.398762,24.313800,25.831385,23.762915,21.694445,66.386573
2,2020-02-10,26.561476,26.568664,25.368510,25.387675,61624400,23.642047,24.659596,26.274288,23.970366,21.666444,70.134117
3,2020-02-11,26.580639,27.277735,26.369834,26.997458,96885200,23.868093,24.955141,26.676574,24.139609,21.602644,70.208247
4,2020-02-12,27.359182,27.392719,26.853728,27.042974,60804000,24.136638,25.324994,27.192868,24.361314,21.529761,73.126626


In the code, the reason for changing the column names is that lightweight_charts demands a specific naming structure to plot the data. Now that we have adequate data to work with, let’s make some cool visualizations.

In [5]:
if __name__ == '__main__':

    chart = Chart()
    chart.set(df)
    chart.show(block = True)

It just takes as little as three lines of code to create a graph in the look of TradingView’s charting platform. And the code is very straightforward in nature. We are first creating an instance of the class Chart and assigned it to the chart variable. Then using the set function, we are setting the data of the chart. Finally, we are displaying the created chart with the help of the show function. 

This output is absolutely stunning for a program of three lines of code. But people who are disappointed at the output after viewing the thumbnail of this article, don't worry! Because now, we are going to up the game by customizing the whole theme of the plot, adding more details, and simply, making it even more beautiful. Here’s the code to do that:

In [6]:
if __name__ == '__main__':

    chart = Chart(title="AAPL Stock Price", height = 600, width = 1000)

    chart.grid(vert_enabled = True, horz_enabled = True)

    chart.layout(background_color='#131722', font_family='Trebuchet MS', font_size = 16)

    chart.candle_style(up_color='#2962ff', down_color='#e91e63',
                    border_up_color='#2962ffcb', border_down_color='#e91e63cb',
                    wick_up_color='#2962ffcb', wick_down_color='#e91e63cb')

    chart.volume_config(up_color='#2962ffcb', down_color='#e91e63cb')

    chart.legend(visible = True, font_family = 'Trebuchet MS', ohlc = True, percent = True)

    #####################################################################################

    chart.set(df)

    # Create line series for EMAs
    ema12_line = chart.create_line('EMA 12', color='#ffeb3b', width=1, price_label=True)
    ema12_line.set(df[['Date', 'EMA 12']])

    ema25_line = chart.create_line('EMA 25', color='#26c6da', width=1, price_label=True)
    ema25_line.set(df[['Date', 'EMA 25']])

    chart.show(block = True)

The code might not be as short as the previous one for the basic plot, but it’s actually very simple. And for easy explanation, I’ve divided the code into two separate parts. The first part is about theme customization. It includes changing the background color, the colors of the candles and volume bars, and so on. Basically, the things related to the style of the plot are dealt with in the first part. The second part is about adding details to the plot. 

This is absolutely fantastic! We fully customized the whole look and feel of the plot and added more details like SMA lines and legends for a more insightful graph.


Now let’s move our focus from historical graphs to another cool feature provided by the lightweight_charts library which is the real-time charting feature. Real-time charts are extremely useful for day traders to keep track of the latest price movements and TradingView is most preferred for such charts. Just like how we replicated the historical charts of TradingView, let’s do the same thing for real-time charts too. This is the code to create a real-time TradingView chart:

In this code, we are not actually using the real-time data of stock prices but rather simulating it using the previously extracted historical data. We are first splitting the historical data into two separate dataframes. The first one is used as the initial data for the plot and the second one is used as the real-time data which is done by updating the data points of the plot with the help of a for-loop. 

Pretty cool, right?! But like how there was a lot of scope for improvements in the basic historical graph, this real-time chart can also be improved and modified in a lot of places. We can first change the theme of the plot and similar to how we added SMA lines to the historical chart for better insights, we can add more details for an informative visualization. Here’s the code for the modified or advanced version of the initial real-time chart:

In [7]:
from lightweight_charts import Chart
import pandas as pd

# Assuming `df` is already a pandas DataFrame with 'Date', 'Open', 'High', 'Low', 'Close', 'EMA 12', and 'EMA 25' columns.
# It's good practice to convert the 'Date' column to the correct datetime format.
# df['Date'] = pd.to_datetime(df['Date'])

if __name__ == '__main__':
    rt_chart = Chart()

    # Set the main candlestick data for the chart.
    # The 'lightweight-charts' library expects a DataFrame with columns like 'Date', 'Open', 'High', 'Low', 'Close'.
    rt_chart.set(df)

    # Create line series for EMAs
    ema12_line = rt_chart.create_line('EMA 12', color='#ffeb3b', width=1, price_label=True)
    ema12_line.set(df[['Date', 'EMA 12']])

    ema25_line = rt_chart.create_line('EMA 25', color='#26c6da', width=1, price_label=True)
    ema25_line.set(df[['Date', 'EMA 25']])

    # Initialize a list to hold the markers
    markers = []

    # Iterate through the DataFrame to find crossover points
    for i in range(1, len(df)):
        p_ema12, p_ema25 = df.iloc[i-1]['EMA 12'], df.iloc[i-1]['EMA 25']
        c_ema12, c_ema25 = df.iloc[i]['EMA 12'], df.iloc[i]['EMA 25']
        
        current_time = df.iloc[i]['Date']

        # Check for buy signal (EMA 12 crosses above EMA 25)
        if p_ema12 < p_ema25 and c_ema12 > c_ema25:
            markers.append({
                'time': current_time,
                'position': 'below',
                'shape': 'arrow_up',
                'color': '#33de3d',
                'text': 'Buy'
            })
        
        # Check for sell signal (EMA 12 crosses below EMA 25)
        elif p_ema12 > p_ema25 and c_ema12 < c_ema25:
            markers.append({
                'time': current_time,
                'position': 'above',
                'shape': 'arrow_down',
                'color': '#f485fb',
                'text': 'Sell'
            })

    # Add all markers at once. It's more efficient than adding them individually in a loop.
    if markers:
        rt_chart.marker_list(markers)
    
rt_chart.show(block = True)
